<a href="https://colab.research.google.com/github/agolia14/Coupan-Redemption-Prediction/blob/Data-Exploration/Customer_Transaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from collections import Counter
import pickle
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [2]:
!git init
!git config --global user.email "agolia14@gmail.com"

!git remote add origin https://d0e5c5d2c5891214a6eb9a978deb96f359421b01@github.com/agolia14/Coupan-Redemption-Prediction.git
!git fetch
!git checkout Data-Exploration

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 195 (delta 102), reused 103 (delta 37), pack-reused 0
Receiving objects: 100% (195/195), 11.91 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (102/102), done.
From https://github.com/agolia14/Coupan-Redemption-Prediction
 * [new branch]      Data-Exploration -> origin/Data-Exploration
 * [new branch]      master           -> origin/master
Branch 'Data-Exploration' set up to track remote branch 'Data-Exploration' from 'origin'.
Switched to a new branch 'Data-Exploration'


!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs

## Make it Code Cell Only if GIT is not ready for large files
!git lfs install

!git lfs track "*.npy"

!git add .gitattributes

!git commit -m "Updating git Attributes"

!git push origin

In [3]:
raw = pd.read_csv("data/train/customer_transaction_data.csv", parse_dates=['date'], infer_datetime_format=True)
cust_demographics = np.load("npy/customer_demographics.npy", allow_pickle=True)
item_data = np.load("npy/item_data.npy", allow_pickle=True)
raw.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [4]:
data = pd.concat([raw,
           pd.DataFrame(columns=["day","month","year", "discount"])],
         axis=1)

data.day = [date.day for date in data.date]
data.month = [date.month for date in data.date]
data.year = [date.year-2000 for date in data.date]
data.discount = data.other_discount + data.coupon_discount
data.discount = [(discount != 0) for discount in data.discount]

data.drop(['date', 'quantity', 'selling_price', 'other_discount', 'coupon_discount'], axis=1, inplace=True)
data = data.drop_duplicates()
print("Shape = ", data.shape)
data.head()

Shape =  (1320403, 6)


,customer_id,item_id,day,month,year,discount
0,1501,26830,2,1,12,True
1,1501,54253,2,1,12,True
2,1501,31962,2,1,12,True
3,1501,33647,2,1,12,False
4,1501,48199,2,1,12,True


In [38]:
data_cust = data.copy()#.merge(cust_demographics, how = "left").drop(["customer_id"], axis=1)
data_cust_item = data_cust.merge(item_data, how = "left").drop(["item_id",'customer_id','year'], axis=1)
data_cust_item.fillna(-1, inplace=True)
print("Shape = ", data_cust_item.shape)
data_cust_item.head()

Shape =  (1320403, 27)


,day,month,discount,brand,enc_brand_type,enc_category,brand_Established,brand_Local,Alcohol,Bakery,"Dairy, Juices & Snacks",Flowers & Plants,Fuel,Garden,Grocery,Meat,Miscellaneous,Natural Products,Packaged Meat,Pharmaceutical,Prepared Food,Restauarant,Salads,Seafood,Skin & Hair Care,Travel,Vegetables (cut)
0,2,1,True,56,1,9,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2,1,True,56,1,9,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,1,True,524,0,6,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2,1,False,1134,0,6,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2,1,True,524,0,6,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
data_cust_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320403 entries, 0 to 1320402
Data columns (total 27 columns):
day                       1320403 non-null int64
month                     1320403 non-null int64
discount                  1320403 non-null bool
brand                     1320403 non-null int64
enc_brand_type            1320403 non-null int64
enc_category              1320403 non-null int64
brand_Established         1320403 non-null uint8
brand_Local               1320403 non-null uint8
Alcohol                   1320403 non-null uint8
Bakery                    1320403 non-null uint8
Dairy, Juices & Snacks    1320403 non-null uint8
Flowers & Plants          1320403 non-null uint8
Fuel                      1320403 non-null uint8
Garden                    1320403 non-null uint8
Grocery                   1320403 non-null uint8
Meat                      1320403 non-null uint8
Miscellaneous             1320403 non-null uint8
Natural Products          1320403 non-null uint8
Packag

In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
X = data_cust_item.drop(["discount"], axis=1)
y = data_cust_item.discount
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

In [46]:
model = RandomForestClassifier(random_state=10, max_depth=25, n_estimators=20, min_samples_split=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Score = ", roc_auc_score(y_test, y_pred))
pd.DataFrame([model.feature_importances_], columns=X.columns).transpose().sort_values(by = 0, ascending=False)

Score =  0.6977384016923323


,0
brand,0.548265
day,0.093120
month,0.075067
enc_category,0.066208
Grocery,0.056963
Pharmaceutical,0.050822
Packaged Meat,0.028549
Natural Products,0.022460
Fuel,0.019952
Bakery,0.006076


In [18]:
model = XGBClassifier(random_state=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Score = ", roc_auc_score(y_test, y_pred))
pd.DataFrame([model.feature_importances_], columns=X.columns).transpose()

Score =  0.6293843418727754


,0
day,0.000880
month,0.002236
year,0.004086
rented,0.000000
family_size,0.008198
no_of_children,0.004484
income_bracket,0.006002
enc_age_range,0.007138
enc_marital_status,0.003519
Age_18-25,0.000000


In [50]:
from keras import Sequential
from keras.layers import *

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from tensorflow.metrics import auc
from keras import regularizers
import tensorflow as tf

from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
def create_baseline():
  model = Sequential()
  model.add(Dense(units=60, input_dim=26, kernel_regularizer=regularizers.l2(0.01)))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.add(Dense(units=50))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('softmax')
  model.add(Dense(units=30))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.add(Dense(units=10))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('softmax')
  model.add(Dense(units=1))
  Dropout(0.2)
  model.add(BatchNormalization())
  Activation('relu')
  model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=[roc_auc, "accuracy"])
  
  return model

In [0]:
def roc_auc(y_true, y_pred):
  return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [54]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=500, verbose=1)
estimator.fit(X_train, y_train)

Epoch 1/20
990302/990302 [==============================] - 16s 16us/step - loss: 2.1439 - roc_auc: 0.5466 - acc: 0.4706
Epoch 2/20
990302/990302 [==============================] - 14s 14us/step - loss: 1.0540 - roc_auc: 0.5441 - acc: 0.4998
Epoch 3/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.8079 - roc_auc: 0.5924 - acc: 0.5703
Epoch 4/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.6617 - roc_auc: 0.6236 - acc: 0.6108
Epoch 5/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.6574 - roc_auc: 0.6242 - acc: 0.6130
Epoch 6/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.6552 - roc_auc: 0.6250 - acc: 0.6135
Epoch 7/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.6539 - roc_auc: 0.6266 - acc: 0.6139
Epoch 8/20
990302/990302 [==============================] - 14s 14us/step - loss: 0.6532 - roc_auc: 0.6264 - acc: 0.6143
Epoch 9/20
990302/990302 [======

In [55]:
pred = estimator.predict(X_test)

print("Score = ", roc_auc_score(y_test, y_pred))

#pred = pd.DataFrame(pred, columns=['redemption_status'])
#idx = pd.DataFrame(test_campaign_coupItem.id)
#y_pred = pd.concat([idx, pred], axis=1)
#y_pred.to_csv("Submission_NN.csv", index=False)

330101/330101 [==============================] - 1s 4us/step
Score =  0.6977384016923323
